In [ ]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor

from sklearn import metrics
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import shapiro
from sklearn import preprocessing

from warnings import filterwarnings
filterwarnings('ignore')

from sklearn import metrics
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/real-estate-dataset/data.csv")

df = data.copy()
df.head()

In [ ]:
def Missing_Values(data):
    variable_name=[]
    total_value=[]
    total_missing_value=[]
    missing_value_rate=[]
    unique_value_list=[]
    total_unique_value=[]
    data_type=[]
    for col in data.columns:
        variable_name.append(col)
        data_type.append(data[col].dtype)
        total_value.append(data[col].shape[0])
        total_missing_value.append(data[col].isnull().sum())
        missing_value_rate.append(round(data[col].isnull().sum()/data[col].shape[0],3))
        unique_value_list.append(data[col].unique())
        total_unique_value.append(len(data[col].unique()))
    missing_data=pd.DataFrame({"Variable":variable_name,"Total_Value":total_value,\
                             "Total_Missing_Value":total_missing_value,"Missing_Value_Rate":missing_value_rate,
                             "Data_Type":data_type,"Unique_Value":unique_value_list,\
                               "Total_Unique_Value":total_unique_value})
    return missing_data.sort_values("Missing_Value_Rate",ascending=False)


a quick look at the data

In [ ]:
Missing_Values(df)

In [ ]:
df.describe().T

In [ ]:
df.drop("CHAS", axis = 1,inplace = True)

#### Handling outlier

Extreme Cleaning:
lower_threshold: Below the first quartile (Q1) − 3 ∗ interquartile range (IQR)
upper_threshold: Above the third quartile (Q3) + 3 ∗ interquartile range (IQR)
    
Standard Cleaning:
lower_threshold: Below the first quartile (Q1) − 1.5 ∗ interquartile range (IQR)
upper_threshold: Above the third quartile (Q3) + 1.5 ∗ interquartile range (IQR)
where 'IQR' is equal to [Q3-Q1]=[third_quartile - first_quartile]. This technique has disadvantages while dealing with sparse columns, since taking the middle lenght (75%-25%) may bring zero distanced IQR.


In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3-Q1
IQR

In [ ]:
minn = Q1 - 3 * IQR
maxx = Q3 + 3 * IQR
out_lier = df[(df < minn)  | (df > maxx)].count()
out_lier

In [ ]:
outlier_ratio  = out_lier/df.count()
outlier_ratio

In [ ]:
df_out_lier_cleaned = df[~((df < minn)  | (df > maxx)).any(axis=1)]
df_out_lier_cleaned

In [ ]:
Missing_Values(df_out_lier_cleaned)

Fill na values

In [ ]:
df_out_lier_cleaned["RM"] = df_out_lier_cleaned["RM"].fillna((df_out_lier_cleaned["RM"].median()))

In [ ]:
Missing_Values(df_out_lier_cleaned)

In [ ]:
df_out_lier_cleaned.corr()

Let's apply train test split to data

In [ ]:
X = df_out_lier_cleaned.drop("CRIM", axis = 1)
y = df_out_lier_cleaned["CRIM"]
X = sm.add_constant(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

x_train_shape = X_train.shape
x_test_shape = X_test.shape
y_train_shape = y_train.shape
y_test_shape = y_test.shape

print("X train shape: {}\nX test shape: {}\n\ny train shape: {}\ny test shape: {}".format(x_train_shape, x_test_shape, y_train_shape, y_test_shape))

# Linear Regression Assumptions

## * Linearity 
## * Normality of the Error Terms 
## * No Multicollinearity among Predictors 
## * No Autocorrelation of the Error Terms
## * Homoscedasticity

In [ ]:
lin_reg = LinearRegression()
lin_reg_model = lin_reg.fit(X_train, y_train)

y_pred = lin_reg_model.predict(X_test)


test_set_rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))
test_set_r2 = r2_score(y_test, y_pred)

print("R2: {}\nRMSE: {}".format(test_set_r2, test_set_rmse))

### Linearity

In [ ]:
sns.pairplot(df_out_lier_cleaned, kind = "reg")

### Normality of the Error Terms

In [ ]:
def normal_errors_assumption(model, features, label, p_value_thresh=0.05):
    """
    Normality: Assumes that the error terms are normally distributed. If they are not,
    nonlinear transformations of variables may solve this.
               
    This assumption being violated primarily causes issues with the confidence intervals
    """
    from statsmodels.stats.diagnostic import normal_ad
    print('Assumption 2: The error terms are normally distributed', '\n')
    
    # Calculating residuals for the Anderson-Darling test
    df_results = calculate_residuals(model, features, label)
    
    print('Using the Anderson-Darling test for normal distribution')

    # Performing the test on the residuals
    p_value = normal_ad(df_results['Residuals'])[1]
    print('p-value from the test - below 0.05 generally means non-normal:', p_value)
    
    # Reporting the normality of the residuals
    if p_value < p_value_thresh:
        print('Residuals are not normally distributed')
    else:
        print('Residuals are normally distributed')
    
    # Plotting the residuals distribution
    plt.subplots(figsize=(12, 6))
    plt.title('Distribution of Residuals')
    sns.distplot(df_results['Residuals'])
    plt.show()
    
    print()
    if p_value > p_value_thresh:
        print('Assumption satisfied')
    else:
        print('Assumption not satisfied')
        print()
        print('Confidence intervals will likely be affected')
        print('Try performing nonlinear transformations on variables')
        
        
        
def calculate_residuals(model, features, label):
    """
    Creates predictions on the features with the model and calculates residuals
    """
    predictions = model.predict(features)
    df_results = pd.DataFrame({'Actual': label, 'Predicted': predictions})
    df_results['Residuals'] = abs(df_results['Actual']) - abs(df_results['Predicted'])
    
    return df_results        

In [ ]:
normal_errors_assumption(lin_reg_model, X_train, y_train)

### No Multicollinearity among Predictors

In [ ]:
plt.figure(figsize = (10,8))
sns.heatmap(pd.DataFrame(df_out_lier_cleaned).corr(), annot=True)
plt.title('Correlation of Variables')
plt.show()

In [ ]:
def variance_inflation_factor(exog, exog_idx):
    """
    exog : ndarray, (nobs, k_vars)
        design matrix with all explanatory variables, as for example used in
        regression
    exog_idx : int
        index of the exogenous variable in the columns of exog
    """
    k_vars = exog.shape[1]
    x_i = exog[:, exog_idx]
    mask = np.arange(k_vars) != exog_idx
    x_noti = exog[:, mask]
    r_squared_i = sm.OLS(x_i, x_noti).fit().rsquared
    vif = 1. / (1. - r_squared_i)
    return vif

def calculate_vif_(X, thresh=10.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]

In [ ]:
calculate_vif_(X_train)

In [ ]:
X = df_out_lier_cleaned.drop(["CRIM","RM", "B", "NOX", "PTRATIO", "TAX", "AGE"], axis = 1)
y = df_out_lier_cleaned["CRIM"]
#X = sm.add_constant(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

x_train_shape = X_train.shape
x_test_shape = X_test.shape
y_train_shape = y_train.shape
y_test_shape = y_test.shape

print("X train shape: {}\nX test shape: {}\n\ny train shape: {}\ny test shape: {}".format(x_train_shape, x_test_shape, y_train_shape, y_test_shape))

In [ ]:
ols_model = sm.OLS(y_train, X_train)
ols_results = ols_model.fit()
print(ols_results.summary())

Dropped ZN and INDUS because P value bigger than 0.05 so they do not contribute significantly to the model

In [ ]:
X = df_out_lier_cleaned.drop(["CRIM","RM", "B", "NOX", "PTRATIO", "TAX", "AGE", "INDUS", "ZN"], axis = 1)
y = df_out_lier_cleaned["CRIM"]
#X = sm.add_constant(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

x_train_shape = X_train.shape
x_test_shape = X_test.shape
y_train_shape = y_train.shape
y_test_shape = y_test.shape

print("X train shape: {}\nX test shape: {}\n\ny train shape: {}\ny test shape: {}".format(x_train_shape, x_test_shape, y_train_shape, y_test_shape))

In [ ]:
ols_model = sm.OLS(y_train, X_train)
ols_results = ols_model.fit()
y_pred = ols_results.predict(X_test)

# calc rmse
ols_rmse = rmse(y_test, y_pred)

print(ols_results.summary())

In [ ]:
ols_rmse

### No Autocorrelation of the Error Terms

In [ ]:
def autocorrelation_assumption(model, features, label):
    """
    Autocorrelation: Assumes that there is no autocorrelation in the residuals. If there is
                     autocorrelation, then there is a pattern that is not explained due to
                     the current value being dependent on the previous value.
                     This may be resolved by adding a lag variable of either the dependent
                     variable or some of the predictors.
    """
    from statsmodels.stats.stattools import durbin_watson
    print('Assumption 4: No Autocorrelation', '\n')
    
    # Calculating residuals for the Durbin Watson-tests
    df_results = calculate_residuals(model, features, label)

    print('\nPerforming Durbin-Watson Test')
    print('Values of 1.5 < d < 2.5 generally show that there is no autocorrelation in the data')
    print('0 to 2< is positive autocorrelation')
    print('>2 to 4 is negative autocorrelation')
    print('-------------------------------------')
    durbinWatson = durbin_watson(df_results['Residuals'])
    print('Durbin-Watson:', durbinWatson)
    if durbinWatson < 1.5:
        print('Signs of positive autocorrelation', '\n')
        print('Assumption not satisfied')
    elif durbinWatson > 2.5:
        print('Signs of negative autocorrelation', '\n')
        print('Assumption not satisfied')
    else:
        print('Little to no autocorrelation', '\n')
        print('Assumption satisfied')

In [ ]:
autocorrelation_assumption(ols_results, X_train, y_train)

### Homoscedasticity

In [ ]:
def homoscedasticity_assumption(model, features, label):
    """
    Homoscedasticity: Assumes that the errors exhibit constant variance
    """
    print('Assumption 5: Homoscedasticity of Error Terms', '\n')
    
    print('Residuals should have relative constant variance')
        
    # Calculating residuals for the plot
    df_results = calculate_residuals(model, features, label)

    # Plotting the residuals
    plt.subplots(figsize=(12, 6))
    ax = plt.subplot(111)  # To remove spines
    plt.scatter(x=df_results.index, y=df_results.Residuals, alpha=0.5)
    plt.plot(np.repeat(0, df_results.index.max()), color='darkorange', linestyle='--')
    ax.spines['right'].set_visible(False)  # Removing the right spine
    ax.spines['top'].set_visible(False)  # Removing the top spine
    plt.title('Residuals')
    plt.show()  

In [ ]:
homoscedasticity_assumption(ols_results, X_train, y_train)

**We have to solve the  normal errorr assumption**

In [ ]:
normal_errors_assumption(ols_results, X_train, y_train)

Some functions belong to **Jeff Macaluso**


I wonder your thoughts about the notebook

The notebook will be updated in the future